In [21]:
from bs4 import BeautifulSoup
import requests
from requests import get
import pandas as pd
import numpy as np
import re
import dateutil.parser

In [2]:
url_list= ['https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2020-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&ref_=adv_prv',
           'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2020-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&start=251&ref_=adv_nxt',
           'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2020-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&start=501&ref_=adv_nxt', 
           'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2020-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&start=751&ref_=adv_nxt', 
           'https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2020-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&start=1001&ref_=adv_nxt']

In [34]:
url_list= ['https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2020-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&start=1501&ref_=adv_nxt']
           
def getmovielinks(url_list):
    movie_links=[]
    for url in url_list:
        response=requests.get(url)
        response.status_code
        soup=BeautifulSoup(response.text, 'lxml')
        x = soup.find_all('div', class_="lister-item-content")
        for box in x:
            movie_links.append(box.find('a').get('href'))

    return movie_links

movielinks=getmovielinks(url_list)

In [35]:
movielinks

['/title/tt0463998/',
 '/title/tt1647668/',
 '/title/tt0978764/',
 '/title/tt0388500/',
 '/title/tt1590193/',
 '/title/tt0384793/',
 '/title/tt0452608/',
 '/title/tt0443632/',
 '/title/tt3799694/',
 '/title/tt6802308/',
 '/title/tt1961175/',
 '/title/tt4560436/',
 '/title/tt1762399/',
 '/title/tt0162983/',
 '/title/tt0398017/',
 '/title/tt0286788/',
 '/title/tt0247425/',
 '/title/tt0838232/',
 '/title/tt4244998/',
 '/title/tt3882082/',
 '/title/tt0399201/',
 '/title/tt0271668/',
 '/title/tt2083383/',
 '/title/tt0265349/',
 '/title/tt0824747/',
 '/title/tt0301470/',
 '/title/tt0475394/',
 '/title/tt1583420/',
 '/title/tt1440728/',
 '/title/tt1860213/',
 '/title/tt0206634/',
 '/title/tt0267891/',
 '/title/tt6722030/',
 '/title/tt6850820/',
 '/title/tt1298644/',
 '/title/tt2935510/',
 '/title/tt3181822/',
 '/title/tt0425430/',
 '/title/tt1298649/',
 '/title/tt0138524/',
 '/title/tt1142977/',
 '/title/tt0158622/',
 '/title/tt1939659/',
 '/title/tt0481369/',
 '/title/tt0267626/',
 '/title/t

In [7]:
movie_urls=[]
def imdbmovies(movielinks):
    for link in movielinks:
        if "title" in link:
            final='https://www.imdb.com'+link
            movie_urls.append(final)
            
    return movie_urls

In [37]:
def get_movie_value(soup, field_name):
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [6]:
response = requests.get('https://www.imdb.com/title/tt0265666/')
response.status_code

200

In [53]:
soup = BeautifulSoup(response.text, 'lxml')

In [54]:
title = soup.find("title").text[:-14].strip()
title

'The Royal Tenenbaums'

In [55]:
release_date = soup.find("a", title="See more release dates").text
release_date = release_date.split('(')[0].strip()
release_date = dateutil.parser.parse(release_date)
release_date

datetime.datetime(2002, 1, 4, 0, 0)

In [64]:
metacritic_score = soup.find("div", class_="titleReviewBarItem").findNext("span").text

'76'

In [15]:
director = get_movie_value(soup,'Director:')
director

'Wes Anderson'

In [16]:
actors = []
actor1 = soup.find("h4", class_="inline", text="Stars:").findNext().text
actor2 = soup.find("h4", class_="inline", text="Stars:").findNext().findNext().text
actor3 = soup.find("h4", class_="inline", text="Stars:").findNext().findNext().findNext().text

actors = [actor1] + [actor2] + [actor3]
actors

['Gene Hackman', 'Gwyneth Paltrow', 'Anjelica Huston']

In [19]:
headers = ['movie_title', 'release_date', 'metacritic_score', 'director', 'actors']

tenenbaums_imdb_dict = dict(zip(headers, [title,
                                release_date,
                                metacritic_score,
                                director, 
                                actors]))

imdb_list = [tenenbaums_imdb_dict]
imdb_list

[{'movie_title': 'The Royal Tenenbaums',
  'release_date': datetime.datetime(2002, 1, 4, 0, 0),
  'metacritic_score': '76',
  'director': 'Wes Anderson',
  'actors': ['Gene Hackman', 'Gwyneth Paltrow', 'Anjelica Huston']}]

In [36]:
def get_imdbmovie_dict(link):
    
    base_url = 'https://www.imdb.com'
    
    url = base_url + link
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    headers = ['movie_title', 'release_date', 'criticscore','director', 'actors']
    
    #Get movie title
    try: 
        title = soup.find("title").text[:-14].strip()
    except Exception as error:
        title.append(np.nan) 
            
    #Get release date
    try: 
        release_date = soup.find("a", title="See more release dates").text
        release_date = release_date.split('(')[0].strip()
        release_date = dateutil.parser.parse(release_date)
    except Exception as error:
        release_date.append(np.nan) 
        
    #Get metacritic score
    try:
        metacritic_score = soup.find("div", class_="titleReviewBarItem").findNext("span").text
    except Exception as error:
        metacritic_score.append(np.nan) 
        
    #Get director
    try:
        director = get_movie_value(soup,'Director:')
    except Exception as error:
        director.append(np.nan) 
        
    #Get actors
    try:
        actors = []
        actor1 = soup.find("h4", class_="inline", text="Stars:").findNext().text
        actor2 = soup.find("h4", class_="inline", text="Stars:").findNext().findNext().text
        actor3 = soup.find("h4", class_="inline", text="Stars:").findNext().findNext().findNext().text
        actors = [actor1] + [actor2] + [actor3]
    except Exception as error:
        actors.append(np.nan) 
        
    #Create movie dictionary and return
    imdb_movie_dict = dict(zip(headers, [title, release_date, metacritic_score, director, actors]))

    return imdb_movie_dict

In [38]:
imdb_movies_list = []
import time

for link in movielinks:
    imdb_movies_list.append(get_imdbmovie_dict(link))
    time.sleep(1)

In [39]:
imdb_movies_list

[{'movie_title': 'Freedom Writers',
  'release_date': datetime.datetime(2007, 1, 5, 0, 0),
  'criticscore': '64',
  'director': 'Richard LaGravenese',
  'actors': ['Hilary Swank', 'Imelda Staunton', 'Patrick Dempsey']},
 {'movie_title': 'Million Dollar Arm',
  'release_date': datetime.datetime(2014, 5, 16, 0, 0),
  'criticscore': '56',
  'director': 'Craig Gillespie',
  'actors': ['Jon Hamm', 'Aasif Mandvi', 'Alan Arkin']},
 {'movie_title': 'Sucker Punch',
  'release_date': datetime.datetime(2011, 3, 25, 0, 0),
  'criticscore': '33',
  'director': 'Zack Snyder',
  'actors': ['Emily Browning', 'Vanessa Hudgens', 'Abbie Cornish']},
 {'movie_title': 'Beauty Shop',
  'release_date': datetime.datetime(2005, 3, 30, 0, 0),
  'criticscore': '53',
  'director': 'Bille Woodruff',
  'actors': ['Queen Latifah', 'Alicia Silverstone', 'Djimon Hounsou']},
 {'movie_title': 'The Commuter',
  'release_date': datetime.datetime(2018, 1, 12, 0, 0),
  'criticscore': '56',
  'director': 'Jaume Collet-Serra',

In [341]:
response = requests.get('https://www.boxofficemojo.com/title/tt0265666/credits/?ref_=bo_tt_tab#tabs') #getting box office mojo stats
response.status_code

200

In [342]:
soup = BeautifulSoup(response.text, 'lxml')

In [343]:
movie_title = soup.find('title').text[:-18]
movie_title

'The Royal Tenenbaums'

In [399]:
rating = get_movie_value(soup,'MPAA')
rating

'R'

In [345]:
dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text.replace("$", "").replace(",", "")
dtg

'52364010'

In [346]:
itg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[1].text.replace("$", "").replace(",", "")
itg

'19080472'

In [347]:
domestic_opening = soup.find_all("span", class_="money")[3].text.replace("$", "").replace(",", "")
domestic_opening

'276981'

In [348]:
budget = soup.find_all("span", class_="money")[4].text.replace("$", "").replace(",", "")
budget

'21000000'

In [349]:
runtime = soup.find(text = "Running Time").findNext().text.split()
runtime = int(runtime[0])*60 + int(runtime[2])
runtime

110

In [350]:
genres = soup.find(text ="Genres").findNext().text.replace("\n", "").split()
genres 

['Comedy', 'Drama']

In [362]:
#testing merging two dataframes together with one movie

headers = ['movie_title', 'runtime', 'genres', 'rating', 'domestic_box_office', "international_box_office",
              'domestic_opening_weekend', 'budget']

tenenbaums_mojo_dict = dict(zip(headers, [movie_title,
                                runtime,
                                genres,
                                rating, 
                                dtg, 
                                itg, 
                                domestic_opening, 
                                budget]))

mojo_list = [tenenbaums_mojo_dict] 
mojo_list

[{'movie_title': 'The Royal Tenenbaums',
  'runtime': 110,
  'genres': ['Comedy', 'Drama'],
  'rating': 'R',
  'domestic_box_office': '52364010',
  'international_box_office': '19080472',
  'domestic_opening_weekend': '276981',
  'budget': '21000000'}]

In [367]:
tenenbaums_mojo_df =pd.DataFrame(mojo_list)
tenenbaums_mojo_df.set_index('movie_title', inplace=True)
tenenbaums_mojo_df

,runtime,genres,rating,domestic_box_office,international_box_office,domestic_opening_weekend,budget
movie_title,,,,,,,
The Royal Tenenbaums,110,"[Comedy, Drama]",R,52364010,19080472,276981,21000000


In [368]:
tenenbaums_test_df = pd.DataFrame(imdb_list)  
tenenbaums_test_df.set_index('movie_title', inplace=True)
tenenbaums_test_df

,release_date,metacritic_score,director,actors
movie_title,,,,
The Royal Tenenbaums,2002-01-04,76,Wes Anderson,"[Gene Hackman, Gwyneth Paltrow, Anjelica Huston]"


In [371]:
tenenbaums_test_df = tenenbaums_test_df.merge(tenenbaums_mojo_df,left_index=True, right_index=True)
tenenbaums_test_df

,release_date,metacritic_score,director,actors,runtime_x,genres_x,rating_x,domestic_box_office_x,international_box_office_x,domestic_opening_weekend_x,budget_x,runtime_y,genres_y,rating_y,domestic_box_office_y,international_box_office_y,domestic_opening_weekend_y,budget_y
movie_title,,,,,,,,,,,,,,,,,,
The Royal Tenenbaums,2002-01-04,76,Wes Anderson,"[Gene Hackman, Gwyneth Paltrow, Anjelica Huston]",110,"[Comedy, Drama]",R,52364010,19080472,276981,21000000,110,"[Comedy, Drama]",R,52364010,19080472,276981,21000000


In [40]:
def get_mojomovie_dict(link):
    
    base_url = 'https://www.boxofficemojo.com'
    
    url = base_url + link
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    
    headers = ['movie_title', 'runtime', 'genres', 'rating', 'domestic_box_office', "international_box_office",
              'domestic_opening_weekend', 'budget']
    
    #Get title
    try:
        movie_title = soup.find('title').text[:-18]
    except Exception as error:
        movie_title.append(np.nan) 
    
    #Get runtime
    #runtime = get_movie_value(soup,'Running')
    try:
        runtime = soup.find(text = "Running Time").findNext().text.split()
        runtime = int(runtime[0])*60 + int(runtime[2])
    except Exception as error:
        runtime.append(np.nan) 
    
    #Get genres
    try:
        genres = soup.find(text ="Genres").findNext().text.replace("\n", "").split()
    except Exception as error:
        genres.append(np.nan) 
    
    #Get rating
    try:
        rating = get_movie_value(soup,'MPAA')
    except Exception as error:
        rating.append(np.nan) 
        
    #Get domestic gross
    try:
        dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0]\
                .text.replace("$", "").replace(",", "")
    except Exception as error:
        dtg.append(np.nan) 
        
    #Get international gross
    try:
        itg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[1]\
                .text.replace("$", "").replace(",", "")
    except Exception as error:
        itg.append(np.nan) 
    
    #Get us opening weekend 
    #domestic_opening = get_movie_value(soup,'Opening')
    try:
        domestic_opening = soup.find_all("span", class_="money")[3].text.replace("$", "").replace(",", "")
    except Exception as error:
        domestic_opening.append(np.nan) 
    
    #Get budget
    #domestic_opening = get_movie_value(soup,'Budget')
    try:
        budget = soup.find_all("span", class_="money")[4].text.replace("$", "").replace(",", "")
    except Exception as error:
        budget.append(np.nan) 
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [movie_title,
                                runtime,
                                genres,
                                rating, 
                                dtg, 
                                itg, 
                                domestic_opening, 
                                budget]))

    return movie_dict

In [41]:
my_movies_list = []
import time 

for link in movielinks:
    my_movies_list.append(get_mojomovie_dict(link))
    time.sleep(1)

In [42]:
my_movies_list

[{'movie_title': 'Freedom Writers',
  'runtime': 123,
  'genres': ['Biography', 'Crime', 'Drama'],
  'rating': 'PG-13',
  'domestic_box_office': '36605602',
  'international_box_office': '6489573',
  'domestic_opening_weekend': '9405582',
  'budget': '21000000'},
 {'movie_title': 'Million Dollar Arm',
  'runtime': 124,
  'genres': ['Biography', 'Drama', 'Sport'],
  'rating': 'PG',
  'domestic_box_office': '36457627',
  'international_box_office': '2025828',
  'domestic_opening_weekend': '10515659',
  'budget': '25000000'},
 {'movie_title': 'Sucker Punch',
  'runtime': 110,
  'genres': ['Action', 'Adventure', 'Fantasy'],
  'rating': 'PG-13',
  'domestic_box_office': '36392502',
  'international_box_office': '53400000',
  'domestic_opening_weekend': '19058199',
  'budget': '82000000'},
 {'movie_title': 'Beauty Shop',
  'runtime': 105,
  'genres': ['Comedy'],
  'rating': 'PG-13',
  'domestic_box_office': '36351350',
  'international_box_office': '894103',
  'domestic_opening_weekend': '12

In [43]:
test_imdb_movie_df = pd.DataFrame(imdb_movies_list)
test_imdb_movie_df.set_index('movie_title', inplace=True)

In [44]:
movies_df = pd.DataFrame(my_movies_list)  #convert list of dict to df
movies_df.set_index('movie_title', inplace=True)

In [45]:
movies_df = movies_df.merge(test_imdb_movie_df,left_index=True, right_index=True)

In [46]:
movies_df

,runtime,genres,rating,domestic_box_office,international_box_office,domestic_opening_weekend,budget,release_date,criticscore,director,actors
movie_title,,,,,,,,,,,
Freedom Writers,123,"[Biography, Crime, Drama]",PG-13,36605602,6489573,9405582,21000000,2007-01-05,64,Richard LaGravenese,"[Hilary Swank, Imelda Staunton, Patrick Dempsey]"
Million Dollar Arm,124,"[Biography, Drama, Sport]",PG,36457627,2025828,10515659,25000000,2014-05-16,56,Craig Gillespie,"[Jon Hamm, Aasif Mandvi, Alan Arkin]"
Sucker Punch,110,"[Action, Adventure, Fantasy]",PG-13,36392502,53400000,19058199,82000000,2011-03-25,33,Zack Snyder,"[Emily Browning, Vanessa Hudgens, Abbie Cornish]"
Beauty Shop,105,[Comedy],PG-13,36351350,894103,12801465,12801465,2005-03-30,53,Bille Woodruff,"[Queen Latifah, Alicia Silverstone, Djimon Hou..."
The Commuter,104,"[Action, Mystery, Thriller]",PG-13,36343858,83598529,13701452,36343858,2018-01-12,56,Jaume Collet-Serra,"[Liam Neeson, Vera Farmiga, Patrick Wilson]"
...,...,...,...,...,...,...,...,...,...,...,...
The Hate U Give,133,"[Crime, Drama]",PG-13,29719483,5214526,512035,23000000,2018-10-19,81,George Tillman Jr.,"[Amandla Stenberg, Regina Hall, Russell Hornsby]"
When the Bough Breaks,107,"[Drama, Thriller]",PG-13,29747603,910784,14202323,10000000,2016-09-09,28,Jon Cassar,"[Morris Chestnut, Regina Hall, Romany Malco]"
Did You Hear About the Morgans?,103,"[Comedy, Drama, Romance]",PG-13,29580087,55700163,6616571,58000000,2009-12-18,27,Marc Lawrence,"[Hugh Grant, Sarah Jessica Parker, Sam Elliott]"


In [47]:
movies_df.to_csv('seventhpage.csv')


In [118]:
movies_df["rating"].value_counts()

PG-13    103
R         86
PG        54
G          1
Name: rating, dtype: int64